In [1]:

%matplotlib widget
import matplotlib.pyplot as plt
from polygonal_roadmaps import pathfinding
import networkx as nx
import numpy as np
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Range1d, Plot
from bokeh.plotting import from_networkx
from bokeh.models.graphs import NodesAndLinkedEdges
from bokeh.models import Circle, HoverTool, MultiLine, WheelZoomTool, PanTool
from bokeh.transform import linear_cmap, factor_cmap


output_notebook()



Loading BokehJS ...

In [2]:
G = pathfinding.gen_example_graph(5,3)
pos = nx.get_node_attributes(G, 'pos')

In [3]:
pathfinding.spatial_astar(G, 'a', 'c')

['a', 'b', 'c']

In [4]:
# We could use figure here but don't want all the axes and titles
plot = figure()
#plot.add_tools(WheelZoomTool(), PanTool())

# Create a Bokeh graph from the NetworkX input using nx.spring_layout

graph = from_networkx(G, pos)
plot.renderers.append(graph)

# Set some of the default node glyph (Circle) properties
graph.node_renderer.glyph.update(radius=.1)

# When we hover over nodes, highlight adjecent edges too
graph.inspection_policy = NodesAndLinkedEdges()

plot.add_tools(HoverTool(tooltips="@index"))

show(plot)

In [5]:
cost = pathfinding.compute_cost(G, 'e')
path, temporal_graph = pathfinding.spacetime_astar(G, 'a', 'e', cost, node_constraints=[('b', 1)], limit=10, return_graph=True)
path

['a', 'a', 'b', 'c', 'd', 'e']

In [6]:
t = nx.DiGraph()
t.add_nodes_from(pathfinding.temporal_node_list(G, 3, []))

In [7]:
t.nodes()

NodeView(('nat0', 'nbt0', 'nct0', 'ndt0', 'net0', 'nft0', 'ngt0', 'nht0', 'nat1', 'nbt1', 'nct1', 'ndt1', 'net1', 'nft1', 'ngt1', 'nht1', 'nat2', 'nbt2', 'nct2', 'ndt2', 'net2', 'nft2', 'ngt2', 'nht2'))

In [8]:
cbs = pathfinding.CBS(G, [('a', 'e'), ('e', 'a')], limit=8, max_iter=10_000)
best = cbs.run()
best

Found new best solution at iteration 5.


[['a', 'b', 'c', 'd', 'e'], ['e', 'd', 'h', 'g', 'f', 'b', 'a']]

In [9]:
%load_ext snakeviz

In [10]:
%%snakeviz
cbs = pathfinding.CBS(G, [('b', 'e'), ('e', 'a'), ('a', 'f')], limit=8, max_iter=10_000)
best = cbs.run()


Found new best solution at iteration 85.
 
*** Profile stats marshalled to file '/var/folders/f5/7kx8kghx2_722736y_vzg_cm0000gn/T/tmpadiqsfdq'. 
Embedding SnakeViz in this document...


In [ ]:
# We could use figure here but don't want all the axes and titles
plot = figure()
#plot.add_tools(WheelZoomTool(), PanTool())

# Create a Bokeh graph from the NetworkX input using nx.spring_layout
graph = from_networkx(cbs.root.graph(), nx.multipartite_layout, subset_key='weight', scale=1.8, center=(0,0))
plot.renderers.append(graph)

# Set some of the default node glyph (Circle) properties
graph.node_renderer.glyph.update(size=20, fill_color=linear_cmap('fitness', 'Viridis256', 8, 12))

# Set some edge properties too
graph.edge_renderer.glyph.line_dash = [2,2]

# When we hover over nodes, highlight adjecent edges too
graph.inspection_policy = NodesAndLinkedEdges()

plot.add_tools(HoverTool(tooltips="nc: @weight, f: @fitness, @solution, @label, open: @open"))

show(plot)

In [ ]:
cbs.best.solution

In [ ]:
cbs.iteration_counter

In [ ]:
cbs.duplicate_counter

In [ ]:
cbs2 = pathfinding.CBS(nx.grid_graph((10, 10)), [((0,0), (5,5)), ((5,5),(0,0))], limit=10)

In [ ]:
cbs2.run()